In [2]:
import string
import re
import random
from iexfinance.stocks import Stock
from iexfinance.stocks import get_historical_data
from datetime import datetime
import spacy

# Load the spacy model: nlp, en_core_web_md
nlp = spacy.load("en_core_web_md")
## Using spaCy's entity recogniser

# Define included entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

# Define chitchat_response()
def chitchat_response(message):
    # Call match_rule()
    response, phrase = match_rule(rules, message)
    # Return none is response is "default"
    if response == "default":
        return None
    if '{0}' in response:
        response = response.format(phrase)
    return response

def match_rule(rules, message):
    for pattern, responses in rules.items():
        match = re.search(pattern, message)
        if match is not None:
            response = random.choice(responses)
            var = match.group(1) if '{0}' in response else None
            return response, var
    return "default", None

rules = {'if (.*)': ["Do you really think it's likely that {0}", 
                     'Do you wish that {0}',
                     'What do you think about {0}',
                     'Really--if {0}'], 
         'what can (.*)': ["I'm a robot to help you get information about stock"], 
         'do you remember (.*)': ['Did you think I would forget {0}',
                                  "Why haven't you been able to forget {0}", 
                                  'What about {0}', 
                                  'Yes .. and?']
        }


def send_message(state, pending, message, comp):
    message = message.lower()
    response = chitchat_response(message)
    if response is not None:
        print(response)
        return state, None, comp
    
    if item_idetification(message) == 'historical':
        state = CHOOSE_DATE
        new_state, response, pending_state = policy_rules[(state, 'historical')]
    elif interpret(message) is not None:
        new_state, response, pending_state = policy_rules[(state, interpret(message))]
    elif interpret(message) is None:
        new_state, response, pending_state = state, "Sorry, I can't catch you.", None
    if interpret(message) == 'specify_company':
        comp = company_identification(message)
        response = response.format(comp)
    if interpret(message) == 'specify_item':
        info = Stock(comp,token="pk_910d4c5b9a2d4d379af7260ae8e549f2")
        item = item_idetification(message)
        API_search = None
        if item == 'low' or item == 'high' or item == 'latestPrice' or item == 'volume':
            API_search = info.get_quote()[item]
            response = response.format(item, API_search)
        if item == 'logo':
            API_search = info.get_logo()['url']
            response = response.format(item, API_search)
    if interpret(message) == 'date':
        date = extract_entities(message)['DATE']
        if datetime.strptime(date,'%Y %m %d').weekday() == 0 or datetime.strptime(date,'%Y %m %d').weekday() == 6:
            response = "Sorry. I can only find stock information on weekdays."
        else:
            start = datetime(2015, 1, 1)
            end = datetime(2019, 8, 14)
            API_search = get_historical_data('AAPL', start, end, output_format='pandas',token="pk_910d4c5b9a2d4d379af7260ae8e549f2")
            date_dict = API_search.loc[date]
            date_message = "open price: {0}, high price: {1}, low price: {2}, close price: {3}, total volume: {4}".format(date_dict['open'],date_dict['high'],date_dict['low'],date_dict['close'],date_dict['volume'])
            response = response.format(date_message)
    print(response)
    if pending is not None and interpret(message) == 'number':
        new_state, response, pending_state = policy_rules[pending]
        print(response)        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending, comp

def company_identification(message):
    if 'aapl' in message or 'appl' in message or 'aple' in message:
        return 'AAPL'
    if 'googl' in message or 'gogl' in message:
        return 'GOOGL'
    return None

def item_idetification(message):
    if 'low' in message:
        return 'low'
    if 'high' in message:
        return 'high'
    if 'price' in message:
        return 'latestPrice'
    if 'volume' in message:
        return 'volume'
    if 'logo' in message:
        return 'logo'
    if 'historical' in message:
        return 'historical'
    return None

def interpret(message):  
    if '201' in message:
        return 'date'
    elif item_idetification(message) is not None:
        return 'specify_item'
    elif company_identification(message) is not None:
        return 'specify_company'
    elif 'stock' in message:
        return 'search'
    if any([d in message for d in string.digits]) and '201' not in message:
        return 'number'
    return None

# Define the states
INIT=0
AUTHED=1
CHOOSE_COMPANY=2
CHOOSE_ITEM=3
CHOOSE_DATE=4
DATE_DONE=5

# Define the policy rules
policy_rules = {
    (INIT, "search"): (INIT, "You'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "Perfect, welcome back!", None),
    (AUTHED, "search"): (CHOOSE_COMPANY, "Which company would you like to know about?", None),  
    (CHOOSE_COMPANY, "specify_company"): (CHOOSE_ITEM, "What kind of stock information for {0} would you like to see?", None),
#choose item, the second is to back to choose company
    (CHOOSE_ITEM, "specify_item"): (CHOOSE_ITEM, "Ok! The {0} is {1}", None),
    (CHOOSE_ITEM, "specify_company"): (CHOOSE_ITEM, "What kind of stock information for {0} would you like to see?", None),
#choose historical, the second is to back to choose item, the third is to back to choose company
    (CHOOSE_DATE, "historical"): (DATE_DONE, "Ok! But you have to tell me the date first.", None),
    (CHOOSE_DATE, "specify_item"): (CHOOSE_ITEM, "Ok! The {0} is {1}", None),
    (CHOOSE_DATE, "specify_company"): (CHOOSE_ITEM, "What kind of stock information for {0} would you like to see?", None),
#choose date, the second is to back to choose historical, the third is to back to choose item, the fourth is to back to choose company
    (DATE_DONE, "date"): (DATE_DONE, "Ok! Here you are: {0}", None),
    (DATE_DONE, "historical"): (DATE_DONE, "Ok! But you have to tell me the date first.", None),
    (DATE_DONE, "specify_item"): (CHOOSE_ITEM, "Ok! The {0} is {1}", None),
    (DATE_DONE, "specify_company"): (CHOOSE_ITEM, "What kind of stock information for {0} would you like to see?", None)
}


state = INIT
pending = None
comp = None
info = None
x = 1
while x<= 100:
    message = input()
    state, pending, comp = send_message(state, pending, message, comp)
    x += 1
print ("You are tired. Please take a break.")

'''
Example:
# Send the messages
send_messages([
    "What can you do for me?",
    "I'd like to search for some stock info",
    "555-12345",
    "i need some information about Apple",
    "volume", 
    "what about the lowest price",
    "ok, then what about some information about Google",
    "price",
    "hhhhhhhhhhhhhhhhhhh",
    "i want some historical information",
    "i want some information on 2018 5 6",
    "ok, then what about info on 2019 4 9",
    "ok, then back to AAPL",
    "i wanna knnow the logo of apple",
    "what about the price"
])
'''

what can you do for me?
I'm a robot to help you get information about stock
i need some stock information
You'll have to log in first, what's your phone number?
4695
Perfect, welcome back!
Which company would you like to know about?
apple please
What kind of stock information for AAPL would you like to see?
the lowest price
Ok! The low is None
what about the current price?
Ok! The latestPrice is 204.33
what about the volume
Ok! The volume is None
well, i need some information about google
What kind of stock information for GOOGL would you like to see?
its logo please
Ok! The logo is https://storage.googleapis.com/iex/api/logos/GOOGL.png
what about some historical information 
Ok! But you have to tell me the date first.
2018 5 4
Ok! Here you are: open price: 178.25, high price: 184.25, low price: 178.17, close price: 183.83, total volume: 56201317.0


KeyboardInterrupt: 